<a href="https://colab.research.google.com/github/meowthisisacat/sign-language-detection-using-yolo-v5/blob/main/notebook69f109095e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
!pip install gdown

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1fLtnCkoM7pK3XTYzFFioeYdhzTUlMizw'
output = 'merged.v4i.yolov5pytorch.zip'
gdown.download(url, output)

In [ ]:

%pwd


In [ ]:

%cd yolov5


In [ ]:

# install dependencies as necessary
!pip install -r requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
import zipfile

zip_path="/kaggle/working/merged.v4i.yolov5pytorch.zip"
zip_extract="/kaggle/working/sign_language_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(zip_extract)

In [ ]:
%cd ..
%cd sign_language_dataset
!ls


In [ ]:
%cat data.yaml

In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:

num_classes

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /kaggle/working/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### **Train Custom YOLOv5 Detector**

Next, we'll fire off training!

Here, we are able to pass a number of arguments:

**img:** define input image size

**batch:** determine batch size

**epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)

**data:** set the path to our yaml file

**cfg:** specify our model configuration

**weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)

**name:** result names

**nosave:** only save the final checkpoint

**cache:** cache images for faster training

In [ ]:
!ls /kaggle/working/yolov5/models/  # Check if 'custom_yolov5s.yaml' exists


In [ ]:
!cp "/kaggle/working/sign_language_dataset/data.yaml" /kaggle/working
!cp -r "/kaggle/working/sign_language_dataset/valid" /kaggle/working
!cp -r "/kaggle/working/sign_language_dataset/train" /kaggle/working



In [ ]:
!ls "/kaggle/working/sign_language_dataset/"

In [ ]:
!ls -l "/content/sign_language_dataset/valid"
!ls -la "/content/sign_language_dataset/valid"


In [ ]:
!ls -l "/content/sign_language_dataset/train"
!ls -la "/content/sign_language_dataset/valid"

In [ ]:
import shutil

#Define the source and destination paths
source_path = '/content/sign_language_dataset/train'
destination_path = '/content/'

#Move the directory
shutil.move(source_path, destination_path)

print(f"Directory moved from {source_path} to {destination_path}")

In [ ]:
!ls /kaggle/working/  # Check if 'data.yaml' is in the right place

In [ ]:
%cd yolov5

In [ ]:
# # time its performance
%cd /kaggle/working/yolov5/
!python train.py --img 512 --batch 64 --epochs 40 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results --patience 10 --cache

In [ ]:
import shutil
shutil.make_archive("model", 'zip', "/runs/train/yolov5s_results4")

In [ ]:

# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png


In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:

# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source ../test/images
